# 운동2(런지) 자세 분석 및 피드백

### 수집 데이터 전처리 함수 및 자세 분석 클래스 정의

In [145]:
import json
import numpy as np
from tensorflow.keras.models import load_model
from stgcn_sport2 import STGCN_sport2
from scipy.signal import savgol_filter

keypoints_2 = [f"Point_{i}" for i in [0,7,11,12,23,24,25,26,27,28,29,30,31,32]]

def load_json_skeleton_2(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    num_frames_2 = len(data["frames"])
    num_views_2 = 1

    # (1, T, 1, V, 3)
    X_data = np.zeros((1, num_frames_2, num_views_2, num_joints_2, num_features_2), dtype=np.float32)
    views = ["view3"]

    # 중심이동된 모든 xyz를 수집해서 정규화할 것
    all_xyz = []

    for frame_idx, frame in enumerate(data["frames"]):
        for view_idx, view in enumerate(views):
            pts = frame.get(view, {}).get("pts", {})

            # 기준점: Point_23 & Point_24 평균
            if "Point_23" in pts and "Point_24" in pts:
                cx = (pts["Point_23"]["x"] + pts["Point_24"]["x"]) / 2
                cy = (pts["Point_23"]["y"] + pts["Point_24"]["y"]) / 2
                cz = (pts["Point_23"]["z"] + pts["Point_24"]["z"]) / 2
            else:
                cx, cy, cz = 0, 0, 0

            for joint_idx, joint_name in enumerate(keypoints_2):
                if joint_name in pts:
                    x = pts[joint_name]["x"] - cx
                    y = pts[joint_name]["y"] - cy
                    z = pts[joint_name]["z"] - cz
                    X_data[0, frame_idx, view_idx, joint_idx] = [x, y, z]
                    all_xyz.append([x, y, z])

    # (N*T*V*J, 3)
    all_xyz = np.array(all_xyz)
    mean = all_xyz.mean()
    std = all_xyz.std()

    # 전체 평균 기준으로 z-score 정규화
    X_data = (X_data - mean) / std

    return X_data

def predict_multiple_json_skeleton(file_paths):
    results = {}

    for file_path in file_paths:
        try:
            # ✅ JSON 데이터 로드
            X_data = load_json_skeleton_2(file_path)

            # ✅ 모델 예측
            prediction = model.predict(X_data)
            
            # ✅ 예측 결과 처리
            predicted_class = np.argmax(prediction, axis=-1)[0]
            confidence = prediction[0][predicted_class]

            # ✅ 결과 저장
            if predicted_class == 0:
                results[file_path] = f"✅ 올바른 자세 ({confidence * 100:.2f}% 확신)"
            else:
                results[file_path] = f"❌ 잘못된 자세 감지 ({confidence * 100:.2f}% 확신)"

        except Exception as e:
            results[file_path] = f"❌ 예측 실패 (오류: {e})"

    return results
    
class LungePostureAnalyzer:
    def __init__(self, model):
        self.model = model
        self.joint_indices = {
            "head": keypoints_2.index("Point_0"),
            "left_shoulder": keypoints_2.index("Point_11"),
            "right_shoulder": keypoints_2.index("Point_12"),
            "left_hip": keypoints_2.index("Point_23"),
            "right_hip": keypoints_2.index("Point_24"),
            "left_knee": keypoints_2.index("Point_25"),
            "right_knee": keypoints_2.index("Point_26"),
            "left_ankle": keypoints_2.index("Point_27"),
            "right_ankle": keypoints_2.index("Point_28"),
        }

    def detect_faulty_posture(self, skeleton_sequence):
        predictions = self.model.predict(skeleton_sequence)
        predicted_label = np.argmax(predictions, axis=-1)[0]
        confidence = predictions[0][predicted_label]

        # ✅ 결과 저장
        if predicted_label == 0:
            result = f"✅ 올바른 자세 ({confidence * 100:.2f}% 확신)"
        else:
            result = f"❌ 잘못된 자세 감지 ({confidence * 100:.2f}% 확신)"
                
        faults = {}
        
        # ✅ 2. 뷰 차원이 1이면 squeeze() 적용
        if skeleton_sequence.shape[2] == 1:
            skeleton_sequence = np.squeeze(skeleton_sequence, axis=2)  # (batch, frames, joints, features)
            
        if predicted_label == 1:  # 잘못된 자세로 분류된 경우
            faults["발방향"] = self.check_feet_direction(skeleton_sequence)
            faults["무릎정렬"] = self.check_knees_alignment(skeleton_sequence)
            faults["어깨수평"] = self.check_shoulders_level(skeleton_sequence)
            faults["비틀림"] = self.check_body_twist(skeleton_sequence)
        
        return {k: v for k, v in faults.items() if v is not None}, result

    def calculate_vector_angle(self, v1, v2):
        dot = np.sum(v1 * v2, axis=-1)
        norm = (np.linalg.norm(v1, axis=-1) * np.linalg.norm(v2, axis=-1)) + 1e-6
        cos_angle = np.clip(dot / norm, -1.0, 1.0)
        angle = np.arccos(cos_angle)
        return np.degrees(angle)
        
    def check_feet_direction(self, skeleton_sequence):
        la = self.joint_indices['left_ankle'] 
        ra = self.joint_indices['right_ankle'] 
        ls = self.joint_indices["left_shoulder"]
        rs = self.joint_indices["right_shoulder"]
        left_diff = np.abs(skeleton_sequence[:, :, ls, 0] - skeleton_sequence[:, :, la, 0])
        right_diff = np.abs(skeleton_sequence[:, :, rs, 0] - skeleton_sequence[:, :, ra, 0])
        avg_diff = np.mean(np.array([left_diff, right_diff]))
        if avg_diff > 0.2:
            return "몸과 발의 방향이 일치하지 않습니다. 발의 방향을 정면으로 유지하세요"
        return None

    def check_knees_alignment(self, skeleton_sequence, is_left=True):
        knee_idx = self.joint_indices['left_knee'] if is_left else self.joint_indices['right_knee']
        ankle_idx = self.joint_indices['left_ankle'] if is_left else self.joint_indices['right_ankle']
        
        # 무릎 → 발목 벡터 계산
        knee_to_ankle_vec = skeleton_sequence[:, :, ankle_idx, :] - skeleton_sequence[:, :, knee_idx, :]
        
        # y축 기준으로 벡터의 기울기 (수직 벡터: [0, 1, 0])
        vertical_vec = np.array([0, 1, 0])  # y축 기준 수직 벡터
        angle = self.calculate_vector_angle(knee_to_ankle_vec, vertical_vec)  # 무릎-발목 벡터와 y축 벡터 사이의 각도 계산
        # 각도가 일정 threshold 이상이면, 무릎이 과도하게 기울어진 것으로 판단
        if np.mean(angle) > 45:  # 30도 이상 기울어졌으면 비정상
            return "무릎이 과도하게 기울어졌습니다. 무릎을 수직으로 유지하세요."
        
        return None
        
    def check_shoulders_level(self, skeleton_sequence):
        ls = self.joint_indices["left_shoulder"]
        rs = self.joint_indices["right_shoulder"]
        shoulder_diff = np.abs(skeleton_sequence[:, :, ls, 1] - skeleton_sequence[:, :, rs, 1])
        if np.mean(shoulder_diff) > 0.1:  # 기준값은 데이터 스케일에 따라 조정
            return "어깨 높이가 비대칭입니다. 양쪽 어깨를 수평으로 맞춰주세요."
        return None

    def check_body_twist(self, skeleton_sequence):
        ls = self.joint_indices["left_shoulder"]
        rs = self.joint_indices["right_shoulder"]
        lh = self.joint_indices["left_hip"]
        rh = self.joint_indices["right_hip"]
    
        shoulder_vec = skeleton_sequence[:, :, rs, :] - skeleton_sequence[:, :, ls, :]
        hip_vec = skeleton_sequence[:, :, rh, :] - skeleton_sequence[:, :, lh, :]
    
        # 회전량 측정: cross product의 y 성분
        twist = np.cross(shoulder_vec, hip_vec)[..., 1]  # y축 방향 회전 정도
        if np.mean(np.abs(twist)) > 0.045:
            return "몸통이 비틀어져 있습니다. 정면을 유지하세요."
        return None

    def provide_feedback(self, skeleton_sequence):
        """감지된 자세 오류를 기반으로 실시간 피드백을 제공합니다."""
        faults, result = self.detect_faulty_posture(skeleton_sequence)
        
        if not faults:
            return f"{result}\n측정 불가"
        
        feedback = f"{result}\n다음 사항을 수정하세요: "
        for key, message in faults.items():
            feedback += f"\n- {message}"
        
        return feedback

### 모델 로드

In [147]:
# ST-GCN 모델 로드 (미리 저장된 모델 파일 경로)
num_joints_2 = 14 
num_features_2 = 3
num_classes_2 = 2  
adjacency_matrix_norm_2 = np.load("adjacency_matrix_sport2.npy")

model = STGCN_sport2(num_joints_2, num_features_2, adjacency_matrix_norm_2, num_classes_2)

dummy_input = np.random.rand(1, 10, num_joints_2, num_features_2).astype(np.float32)
model(dummy_input)

model.load_weights("stgcn_model_sport2_C1.weights.h5")

In [148]:
model.summary()

Model: "stgcn_sport2_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ graph_conv_layer_24 (GraphConvLayer) │ ?                           │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ graph_conv_layer_25 (GraphConvLayer) │ ?                           │           8,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_24 (Conv1D)                   │ (1, 10, 128)                │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_25 (Conv1D)                   │ (1, 10, 64)                 │          57,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_50               │ (10, 14, 64)                │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_51               │ (10, 14, 128)               │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_12 (Activation)           │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_12          │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (1, 2)                      │             130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ ?                           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,673,218 (6.38 MB)

 Trainable params: 1,672,450 (6.38 MB)

 Non-trainable params: 768 (3.00 KB)

### 로드된 모델 검증

In [150]:
analyzer = LungePostureAnalyzer(model)
# file_path = "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-7-81.json"
# skeleton_sequence = load_json_skeleton_2(file_path)
# feedback = analyzer.provide_feedback(skeleton_sequence)
# print(feedback)

 
file_paths = [
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-7-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-8-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-7-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-8-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-8-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-9-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-8-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-9-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_14-8-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_14-9-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_16-8-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_16-9-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_16-7-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_14-7-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-7-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-7-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-6-81.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-1-91.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-1-92.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-1-93.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-1-94.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-1-82.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-1-83.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-1-84.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-1-85.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-1-86.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-1-87.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-1-88.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-1-89.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-1-90.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-1-91.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-1-92.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-1-93.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-1-94.json",
]

for file_path in file_paths:
    print(file_path)
    skeleton_sequence = load_json_skeleton_2(file_path)
    feedback = analyzer.provide_feedback(skeleton_sequence)
    print(feedback)

D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-7-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
✅ 올바른 자세 (93.39% 확신)
측정 불가
D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_01-8-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
✅ 올바른 자세 (95.83% 확신)
측정 불가
D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-7-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
✅ 올바른 자세 (96.85% 확신)
측정 불가
D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_02-8-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
✅ 올바른 자세 (59.71% 확신)
측정 불가
D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-8-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
✅ 올바른 자세 (92.25% 확신)
측정 불가
D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_04-9-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
46.40214106386478
❌ 잘못된 자세 감지 (92.65% 확신)
다음 사항을 수정하세요: 
- 무릎이 과도하게 기울어졌습니다. 무릎을 수직으로 유지하세요.
D:/Studying/gradu/013.피트니스자세/1.Training/sport2/R3/body_05-8-81.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
✅ 올바른 자세 (58.03% 확신)
측정 불가
D: